In [ ]:
#Instalamos y actualizamos versiones de chromium y selenium para que funcione correctamente.
!apt update
!apt install chromium-chromedriver
!pip install selenium

In [13]:
#Importamos todo lo que necesitamos para el trabajo.
import json
import time
import requests
import urllib.parse
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [14]:
#Definimos las funciones a las que en el código general llamaremos para obtener ciertos datos concretos
#Para cada función, configuramos el driver para la respectiva url, después accedemos al dato concreto con la ruta XPATH y lo devolvemos.
#En la última función (get_founded()) he añadido un filtro ya que en algunas ocasiones en la web teníamos la fecha con dia mes y año, cosa que no me interesaba.

def get_EPS(link): #Cada funcion requiere la url de la página a visitar.
  chrome_options = Options()
  chrome_options.add_argument("--headless")
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome(options=chrome_options)
  driver.get(link)
  driver.implicitly_wait(5)

  EPS = driver.find_element(By.XPATH, '//*[@id="main"]/div[2]/div[2]/table[1]/tbody/tr[5]/td[2]').text

  return EPS

def get_PER(link):
  chrome_options = Options()
  chrome_options.add_argument("--headless")
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome(options=chrome_options)
  driver.get(link)
  driver.implicitly_wait(5)

  PER = driver.find_element(By.XPATH, '//*[@id="main"]/div[2]/div[2]/table[1]/tbody/tr[6]/td[2]').text

  return PER

def get_dividend(link):
  chrome_options = Options()
  chrome_options.add_argument("--headless")
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome(options=chrome_options)
  driver.get(link)
  driver.implicitly_wait(5)

  dividend = driver.find_element(By.XPATH, '//*[@id="main"]/div[2]/div[2]/table[1]/tbody/tr[8]/td[2]').text

  return dividend

def get_founded(link):
  chrome_options = Options()
  chrome_options.add_argument("--headless")
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome(options=chrome_options)
  driver.get(link)
  driver.implicitly_wait(5)

  founded = driver.find_element(By.XPATH, '//*[@id="main"]/div[3]/div[1]/div[1]/div/div[3]/span[2]').text

  try:
      year_founded = datetime.strptime(founded, "%b %d, %Y")
      return str(year_founded.year)
  except ValueError:
      return founded

In [18]:
#CÓDIGO PRINCIPAL

#Configuramos el driver para la página web a visitar
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://stockanalysis.com/markets/gainers/3y/') #Definimos la url.
driver.implicitly_wait(5)

results=[]  #Creamos una lista vacía donde guardaremos todos los valores que iremos obteniendo para después visualizarlos.
page = 1  #Definimos la variable page para mostrarla en el bucle que haremos posteriormente y así limitar el programa a ciertas páginas.

#Empezamos el bucle que va a contar 25 páginas y después parará.
while page < 25:
  #Creamos la variable next_page para asignarle el botón de siguiente página al driver con el XPATH de este.
  next_page = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="main"]/nav[3]/button[2]')))
  time.sleep(1)

  #Definimos la tabla que contiene todos los valores
  table = driver.find_element(By.XPATH, '//*[@id="main-table"]/tbody')

  #Obtenemos todas las filas de la tabla filtrando por el tag ('tr')
  filas = table.find_elements(By.TAG_NAME, 'tr')

  #Creamos un bucle para iterar sobre las filas y extraer todos los datos de cada una.
  for fila in filas:

      #Obtenemos todas las columnas de la fila de la iteración.
      columnas = fila.find_elements(By.TAG_NAME, 'td')

      #Creamos una lista para ir almacenadno los datos de cada fila.
      datos_fila = []

      #Extraemos el link donde se encuentra la información adicional (EPS, PER...) de cada empresa. Este link lo usamos en las funciones anteriores.
      link_element = columnas[1].find_element(By.TAG_NAME, 'a')
      link = link_element.get_attribute('href')

      #Iteramos un bucle para meter cada dato de cada columna a la lista que contiene los datos de las filas.
      for i, columna in enumerate(columnas):
        datos_fila.append(columna.text)

        #En este paso, creamos un condicional que ejecutará las funciones anteriores que nos darán los datos adicionales.
        #Este lo ejecutamos cuando el indice (la columna procesada) es el 2, ya que en ese se encuentra el link al que queremos acceder.
        if i == 2:

            #Asignamos todas las funciones y las metemos en nuestra lista de cada fila, respectivamente.
            EPS = get_EPS(link)
            PER = get_PER(link)
            dividend = get_dividend(link)
            founded = get_founded(link)
            datos_fila.append(link)
            datos_fila.append(EPS)
            datos_fila.append(PER)
            datos_fila.append(dividend)
            datos_fila.append(founded)

      #Cuando tenemos todos los valores de una fila, los metemos en la lista inicial 'results' y vamos a por la siguiente fila de la página.
      results.append(datos_fila)

  #Una vez acabamos todas las filas de la página, se ejecutan estos comandos que pasarán de página (haciendo click en el botón asignado en la variable next_page).
  driver.execute_script("arguments[0].scrollIntoView();", next_page)
  driver.execute_script("arguments[0].click();", next_page)

  #Sumamos uno al contador de página y mostramos el resultado a cada iteración para saber por que página vamos.
  page += 1
  print(page)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [ ]:
#Teniendo el código anterior ejecutado correctamente, podemos descarganos el parquet y el csv de nuestro DataFrame resultante y trabajar con él.
df = pd.DataFrame(results)
df.columns = df.columns.astype(str)

#Guardar el DataFrame como un archivo Parquet y lo descarga
df.to_parquet('archivo_resultados.parquet')
from google.colab import files
files.download('archivo_resultados.parquet')


#Guardar el DataFrame como un archivo csv y lo descarga
pd.DataFrame(results).to_csv('archivo_resultados.csv', index=False)
from google.colab import files
files.download('archivo_resultados.csv')

In [27]:
#Una vez realizado el paso anterior, podemos importar el archivo que contiene el dataframe y trabajar con él.
archivo_csv = '/content/archivo_resultados.csv'
df = pd.read_csv(archivo_csv)
df

,0,1,2,3,4,5,6,7,8,9,10,11
0,1,KGEI,Kolibri Global Energy Inc.,https://stockanalysis.com/stocks/kgei/,0.66,5.82,NaN,2008,"10,436.39%",3.91,"14,602",139.25M
1,2,ATLX,Atlas Lithium Corporation,https://stockanalysis.com/stocks/atlx/,-3.71,NaN,NaN,2011,"3,361.11%",31.29,"98,972",346.49M
2,3,AMR,"Alpha Metallurgical Resources, Inc.",https://stockanalysis.com/stocks/amr/,50.00,6.95,$2.00 (0.59%),2016,"2,889.01%",338.23,"95,522",4.48B
3,4,APLD,Applied Digital Corporation,https://stockanalysis.com/stocks/apld/,-0.57,NaN,NaN,2001,"2,538.24%",6.85,"3,663,687",725.72M
4,5,IVT,InvenTrust Properties Corp.,https://stockanalysis.com/stocks/ivt/,0.04,644.25,$0.86 (3.37%),2005,"2,242.73%",25.55,"90,861",1.73B
...,...,...,...,...,...,...,...,...,...,...,...,...
475,476,GPRE,Green Plains Inc.,https://stockanalysis.com/stocks/gpre/,-2.41,NaN,NaN,2004,89.00%,25.23,"626,438",1.50B
476,477,LECO,"Lincoln Electric Holdings, Inc.",https://stockanalysis.com/stocks/leco/,8.54,25.59,$2.84 (1.31%),1895,88.90%,217.66,"178,301",12.44B
477,478,CAH,"Cardinal Health, Inc.",https://stockanalysis.com/stocks/cah/,0.63,160.03,$2.00 (1.99%),1979,88.49%,100.80,"1,093,481",24.84B
478,479,GVA,Granite Construction Incorporated,https://stockanalysis.com/stocks/gva/,0.35,145.60,$0.52 (1.02%),1990,88.41%,50.84,"124,496",2.23B


Visualizando el resultado, podemos ver varias cosas:

1-No tenemos nombres que diferencien a las columnas con sus respectivos valores, sinó números.

2-Tenemos las 4 filas que son bastante irrelevantes para nuestra investigación. Estas aparecen porque están en la página principal de la que realizamos el scrapping.

3-Investigando a fondo, vemos que tenemos en la columna donde están los años, hay algunos nombres o NaN.

In [67]:
#Reasignamos valores a las columnas
df.columns = ['Nº En Ranking', 'Symbolo', 'Nombre', 'Link Info', 'EPS', 'PER', 'Dividends', 'Año fundado', 'Cambio 3 años', 'Market Cap', 'Stock Price', 'Renueve']

#Vemos como queda
df.head(3)

,Nº En Ranking,Symbolo,Nombre,Link Info,EPS,PER,Dividends,Año fundado,Cambio 3 años,Market Cap,Stock Price,Renueve
0,1,KGEI,Kolibri Global Energy Inc.,https://stockanalysis.com/stocks/kgei/,0.66,5.82,NaN,2008.0,"10,436.39%",3.91,"14,602",139.25M
1,2,ATLX,Atlas Lithium Corporation,https://stockanalysis.com/stocks/atlx/,-3.71,nan,NaN,2011.0,"3,361.11%",31.29,"98,972",346.49M
2,3,AMR,"Alpha Metallurgical Resources, Inc.",https://stockanalysis.com/stocks/amr/,50.0,6.95,$2.00 (0.59%),2016.0,"2,889.01%",338.23,"95,522",4.48B


Realizamos la primera corrección y asignamos valores a las columnas

In [53]:
#Definimos las columnas que queremos eliminar
columnas_a_eliminar = ['Cambio 3 años', 'Market Cap', 'Stock Price', 'Renueve']

#Eliminamos las columnas
df_reducido = df.drop(columns=columnas_a_eliminar)

#Vemos como queda
df_reducido.head(3)

,Nº En Ranking,Symbolo,Nombre,Link Info,EPS,PER,Dividends,Año fundado
0,1,KGEI,Kolibri Global Energy Inc.,https://stockanalysis.com/stocks/kgei/,0.66,5.82,NaN,2008.0
1,2,ATLX,Atlas Lithium Corporation,https://stockanalysis.com/stocks/atlx/,-3.71,nan,NaN,2011.0
2,3,AMR,"Alpha Metallurgical Resources, Inc.",https://stockanalysis.com/stocks/amr/,50.0,6.95,$2.00 (0.59%),2016.0


Seguidamente, eliminamos aquellas columnas que no vamos a necesitar en nuestra investigación.


In [70]:
#Seleccionamos solo aquellas filas que no tienen un número concreto en la columna 'Año fundado' y las eliminamos del DataFrame
mask_no_numericos = pd.to_numeric(df_reducido['Año fundado'], errors='coerce').isna()
df_anyosLimpios = df_reducido.dropna(subset=['Año fundado'], axis=0)
df_anyosLimpios.head(10)

,Nº En Ranking,Symbolo,Nombre,Link Info,EPS,PER,Dividends,Año fundado
0,1,KGEI,Kolibri Global Energy Inc.,https://stockanalysis.com/stocks/kgei/,0.66,5.82,NaN,2008.0
1,2,ATLX,Atlas Lithium Corporation,https://stockanalysis.com/stocks/atlx/,-3.71,nan,NaN,2011.0
2,3,AMR,"Alpha Metallurgical Resources, Inc.",https://stockanalysis.com/stocks/amr/,50.0,6.95,$2.00 (0.59%),2016.0
3,4,APLD,Applied Digital Corporation,https://stockanalysis.com/stocks/apld/,-0.57,nan,NaN,2001.0
4,5,IVT,InvenTrust Properties Corp.,https://stockanalysis.com/stocks/ivt/,0.04,644.25,$0.86 (3.37%),2005.0
5,6,DXLG,"Destination XL Group, Inc.",https://stockanalysis.com/stocks/dxlg/,0.48,9.58,NaN,1987.0
6,7,PMN,"ProMIS Neurosciences, Inc.",https://stockanalysis.com/stocks/pmn/,-1.94,nan,NaN,2004.0
7,8,CEIX,CONSOL Energy Inc.,https://stockanalysis.com/stocks/ceix/,19.99,5.07,$2.20 (2.19%),2017.0
8,9,SGML,Sigma Lithium Corporation,https://stockanalysis.com/stocks/sgml/,-1.06,nan,NaN,2011.0
9,10,HDSN,"Hudson Technologies, Inc.",https://stockanalysis.com/stocks/hdsn/,1.13,11.82,NaN,1994.0


Realizamso la eliminación de aquellas filas que no tienen un valor numérico en la columna de 'Año fundado'

In [71]:
#Filtramos por año de creación superior al 2000
df_anyosFiltrados = df_anyosLimpios[df_anyosLimpios['Año fundado'] > 2000]
df_anyosFiltrados.head(10)

,Nº En Ranking,Symbolo,Nombre,Link Info,EPS,PER,Dividends,Año fundado
0,1,KGEI,Kolibri Global Energy Inc.,https://stockanalysis.com/stocks/kgei/,0.66,5.82,NaN,2008.0
1,2,ATLX,Atlas Lithium Corporation,https://stockanalysis.com/stocks/atlx/,-3.71,nan,NaN,2011.0
2,3,AMR,"Alpha Metallurgical Resources, Inc.",https://stockanalysis.com/stocks/amr/,50.0,6.95,$2.00 (0.59%),2016.0
3,4,APLD,Applied Digital Corporation,https://stockanalysis.com/stocks/apld/,-0.57,nan,NaN,2001.0
4,5,IVT,InvenTrust Properties Corp.,https://stockanalysis.com/stocks/ivt/,0.04,644.25,$0.86 (3.37%),2005.0
6,7,PMN,"ProMIS Neurosciences, Inc.",https://stockanalysis.com/stocks/pmn/,-1.94,nan,NaN,2004.0
7,8,CEIX,CONSOL Energy Inc.,https://stockanalysis.com/stocks/ceix/,19.99,5.07,$2.20 (2.19%),2017.0
8,9,SGML,Sigma Lithium Corporation,https://stockanalysis.com/stocks/sgml/,-1.06,nan,NaN,2011.0
11,12,VIST,"Vista Energy, SAB de CV",https://stockanalysis.com/stocks/vist/,3.53,8.25,NaN,2019.0
14,15,BTU,Peabody Energy Corporation,https://stockanalysis.com/stocks/btu/,7.65,3.19,$0.30 (1.24%),2016.0


Ahora, eliminamos todas aquellas filas que tienen año de fundación de la empresa anterior al 2000.

In [72]:
#Eliminamos las empresas que no han pagado dividendos
df_con_dividendos = df_anyosFiltrados[df_anyosFiltrados['Dividends'].notna()]
df_con_dividendos.head(5)

,Nº En Ranking,Symbolo,Nombre,Link Info,EPS,PER,Dividends,Año fundado
2,3,AMR,"Alpha Metallurgical Resources, Inc.",https://stockanalysis.com/stocks/amr/,50.0,6.95,$2.00 (0.59%),2016.0
4,5,IVT,InvenTrust Properties Corp.,https://stockanalysis.com/stocks/ivt/,0.04,644.25,$0.86 (3.37%),2005.0
7,8,CEIX,CONSOL Energy Inc.,https://stockanalysis.com/stocks/ceix/,19.99,5.07,$2.20 (2.19%),2017.0
14,15,BTU,Peabody Energy Corporation,https://stockanalysis.com/stocks/btu/,7.65,3.19,$0.30 (1.24%),2016.0
16,17,PR,Permian Resources Corporation,https://stockanalysis.com/stocks/pr/,0.9,15.21,$0.37 (2.69%),2016.0


En este paso, eliminamos todas aquellas empresas que no hayan pagado dividendos y nos quedamos con las que sí.

In [73]:
#Convertimos la columna 'EPS' a numérica y filtramos por aquellas que tengan un valor EPS positivo.
df_con_dividendos['EPS'] = pd.to_numeric(df_con_dividendos['EPS'], errors='coerce')
df_con_eps_positivo = df_con_dividendos[df_con_dividendos['EPS'] > 0]
df_con_eps_positivo.head(5)

<ipython-input-73-af135a2386c0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_con_dividendos['EPS'] = pd.to_numeric(df_con_dividendos['EPS'], errors='coerce')


,Nº En Ranking,Symbolo,Nombre,Link Info,EPS,PER,Dividends,Año fundado
2,3,AMR,"Alpha Metallurgical Resources, Inc.",https://stockanalysis.com/stocks/amr/,50.00,6.95,$2.00 (0.59%),2016.0
4,5,IVT,InvenTrust Properties Corp.,https://stockanalysis.com/stocks/ivt/,0.04,644.25,$0.86 (3.37%),2005.0
7,8,CEIX,CONSOL Energy Inc.,https://stockanalysis.com/stocks/ceix/,19.99,5.07,$2.20 (2.19%),2017.0
14,15,BTU,Peabody Energy Corporation,https://stockanalysis.com/stocks/btu/,7.65,3.19,$0.30 (1.24%),2016.0
16,17,PR,Permian Resources Corporation,https://stockanalysis.com/stocks/pr/,0.90,15.21,$0.37 (2.69%),2016.0


En este paso, el filtrado lo realizamos en la columna EPS, en este caso, nos quedamos solo con aquellas empresas que tengan un EPS positivo.

In [75]:
#Convertimos la columna 'PER' en números y filtramos por empresas que tenagn PER inferior a 30
df_con_eps_positivo['PER'] = pd.to_numeric(df_con_eps_positivo['PER'], errors='coerce')
df_filtrado_final = df_con_eps_positivo[df_con_eps_positivo['PER'] < 30]
df_filtrado_final.head(10)

<ipython-input-75-3d26a3ecb479>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_con_eps_positivo['PER'] = pd.to_numeric(df_con_eps_positivo['PER'], errors='coerce')


,Nº En Ranking,Symbolo,Nombre,Link Info,EPS,PER,Dividends,Año fundado
2,3,AMR,"Alpha Metallurgical Resources, Inc.",https://stockanalysis.com/stocks/amr/,50.00,6.95,$2.00 (0.59%),2016.0
7,8,CEIX,CONSOL Energy Inc.,https://stockanalysis.com/stocks/ceix/,19.99,5.07,$2.20 (2.19%),2017.0
14,15,BTU,Peabody Energy Corporation,https://stockanalysis.com/stocks/btu/,7.65,3.19,$0.30 (1.24%),2016.0
16,17,PR,Permian Resources Corporation,https://stockanalysis.com/stocks/pr/,0.90,15.21,$0.37 (2.69%),2016.0
23,24,PBF,PBF Energy Inc.,https://stockanalysis.com/stocks/pbf/,21.66,2.05,$1.00 (2.27%),2012.0
24,25,TGLS,Tecnoglass Inc.,https://stockanalysis.com/stocks/tgls/,4.24,10.84,$0.36 (0.78%),2012.0
29,30,METC,"Ramaco Resources, Inc.",https://stockanalysis.com/stocks/metc/,1.44,11.93,$0.55 (3.22%),2017.0
30,31,ESEA,Euroseas Ltd.,https://stockanalysis.com/stocks/esea/,15.82,1.97,$2.00 (6.40%),2006.0
39,40,STNG,Scorpio Tankers Inc.,https://stockanalysis.com/stocks/stng/,11.89,5.05,$1.40 (2.30%),2010.0
50,51,ESOA,Energy Services of America Corporation,https://stockanalysis.com/stocks/esoa/,0.20,28.90,$0.06 (1.03%),2006.0


En este último paso, filtramos por aquellas empresas que tengan un PER inferior a 30.

Llegados a este punto, tenemos el dataframe totalmente limpio con los datos que necesitamos. En este nos quedan 67 empresas válidas para nuestra investigación.

Para determinar el top 10 de nuestrar 67 empreas, deberemos fijarnos en los 3 valores que tenemos EPS, PER y dividendos. Después de investigar a cerca de ellos, podemos realizar el siguiente filtrado:

In [76]:
#Ordenamos el DataFrame final por EPS descendente, PER ascendente y Dividendos descendente.
df_top_10 = df_filtrado_final.sort_values(by=['EPS', 'PER', 'Dividends'], ascending=[False, True, False])
df_top_10.head(10)

,Nº En Ranking,Symbolo,Nombre,Link Info,EPS,PER,Dividends,Año fundado
2,3,AMR,"Alpha Metallurgical Resources, Inc.",https://stockanalysis.com/stocks/amr/,50.00,6.95,$2.00 (0.59%),2016.0
108,109,DAC,Danaos Corporation,https://stockanalysis.com/stocks/dac/,28.79,2.55,$3.20 (4.35%),2006.0
105,106,MPC,Marathon Petroleum Corporation,https://stockanalysis.com/stocks/mpc/,26.77,5.56,$3.30 (2.23%),2011.0
418,419,AMP,"Ameriprise Financial, Inc.",https://stockanalysis.com/stocks/amp/,24.63,15.44,$5.40 (1.42%),2005.0
100,81,UAN,"CVR Partners, LP",https://stockanalysis.com/stocks/uan/,24.38,2.73,$26.62 (40.67%),2011.0
64,65,CHRD,Chord Energy Corporation,https://stockanalysis.com/stocks/chrd/,22.16,7.53,$1.25 (0.75%),2010.0
23,24,PBF,PBF Energy Inc.,https://stockanalysis.com/stocks/pbf/,21.66,2.05,$1.00 (2.27%),2012.0
7,8,CEIX,CONSOL Energy Inc.,https://stockanalysis.com/stocks/ceix/,19.99,5.07,$2.20 (2.19%),2017.0
133,134,FANG,"Diamondback Energy, Inc.",https://stockanalysis.com/stocks/fang/,17.62,8.82,$0.84 (0.54%),2012.0
446,447,PSX,Phillips 66,https://stockanalysis.com/stocks/psx/,16.57,8.05,$4.20 (3.16%),2012.0


De esta forma, extraemos las 10 mejores empresas a las que invertir en 2024.

Podemos asegurar esto porque para realizar toda esta investigación hemos extraído los datos de la siguiente url: 'https://stockanalysis.com/markets/gainers/3y/'

En esa web se encuentran las casi 2000 empresas que mejor se manitenen en los últimos 3 años. Nosotros, hemos conseguido extraer concretamente 481 empresas de esta web y de ahí las 67 empresas válidas y las 10 mejores.

Podríamos ampliar al búsqueda a todas las empresas de la web pero necesitaríamos más tiempo de ejecución y optimización de código.

Por último, descargamos en formato csv el DataFrame con las 10 mejores empresas.

In [ ]:
#Guardamos el top 10 empresas
df_top_10.head(10).to_csv('top_10_empresas.csv')
from google.colab import files
files.download('top_10_empresas.csv')